<a href="https://colab.research.google.com/github/lis-r-barreto/Data-Engineering/blob/main/02_Exploring_Hacker_News_Posts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploring Hacker News Posts

## Introduction

Hacker News is a site started by the startup incubator [Y Combinator](https://www.ycombinator.com/), where user-submitted stories (known as "posts") are voted and commented upon, similar to reddit. Hacker News is extremely popular in technology and startup circles, and posts that make it to the top of Hacker News' listings can get hundreds of thousands of visitors as a result.

We're specifically interested in posts whose titles begin with either Ask HN or Show HN. Users submit Ask HN posts to ask the Hacker News community a specific question. Below is an example:

*Ask HN: How to improve my personal website?*

Likewise, users submit Show HN posts to show the Hacker News community a project, product, or just generally something interesting. Below is an example:

*Show HN: Wio Link  ESP8266 Based Web of Things Hardware Development Platform'*

We'll compare these two types of posts to determine the following:
- Do Ask HN or Show HN receive more comments on average?
- Do posts created at a certain time receive more comments on average?

## Opening and Exploring the Data

You can find the data set [here](https://www.kaggle.com/hacker-news/hacker-news-posts), but note that it has been reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions.

Below are descriptions of the columns:
- id: The unique identifier from Hacker News for the post
- title: The title of the post
- url: The URL that the posts links to, if the post has a URL
- num_points: The number of points the post acquired, calculated as the total number of upvotes minus the total number of downvotes
- num_comments: The number of comments that were made on the post
- author: The username of the person who submitted the post
- created_at: The date and time at which the post was submitted


Let's start by importing the libraries we need and reading the data set into a list of lists and explore first 5 rows.

In [ ]:
from csv import reader

opened_file = open ('hacker_news.csv',encoding = 'utf8')
read_file = reader (opened_file)
hn = list (read_file)

for row in hn[:5]:
    print (row)
    print('\n')

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


['12579008', 'You have two days to comment if you want stem cells to be classified as your own', 'http://www.regulations.gov/document?D=FDA-2015-D-3719-0018', '1', '0', 'altstar', '9/26/2016 3:26']


['12579005', 'SQLAR  the SQLite Archiver', 'https://www.sqlite.org/sqlar/doc/trunk/README.md', '1', '0', 'blacksqr', '9/26/2016 3:24']


['12578997', 'What if we just printed a flatscreen television on the side of our boxes?', 'https://medium.com/vanmoof/our-secrets-out-f21c1f03fdc8#.ietxmez43', '1', '0', 'pavel_lishin', '9/26/2016 3:19']


['12578989', 'algorithmic music', 'http://cacm.acm.org/magazines/2011/7/109891-algorithmic-composition/fulltext', '1', '0', 'poindontcare', '9/26/2016 3:16']




Notice that the first list in the inner lists contains the column headers, and the lists after contain the data for one row. In order to analyze our data, we need to first remove the row containing the column headers. Let's remove that first row.

In [ ]:
headers = hn[0]  #Do not run this cell more than once
hn = hn[1:]

print (headers)
print('\n')
for row in hn[:5]:
    print (row)
    print('\n')

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


['12579008', 'You have two days to comment if you want stem cells to be classified as your own', 'http://www.regulations.gov/document?D=FDA-2015-D-3719-0018', '1', '0', 'altstar', '9/26/2016 3:26']


['12579005', 'SQLAR  the SQLite Archiver', 'https://www.sqlite.org/sqlar/doc/trunk/README.md', '1', '0', 'blacksqr', '9/26/2016 3:24']


['12578997', 'What if we just printed a flatscreen television on the side of our boxes?', 'https://medium.com/vanmoof/our-secrets-out-f21c1f03fdc8#.ietxmez43', '1', '0', 'pavel_lishin', '9/26/2016 3:19']


['12578989', 'algorithmic music', 'http://cacm.acm.org/magazines/2011/7/109891-algorithmic-composition/fulltext', '1', '0', 'poindontcare', '9/26/2016 3:16']


['12578979', 'How the Data Vault Enables the Next-Gen Data Warehouse and Data Lake', 'https://www.talend.com/blog/2016/05/12/talend-and-Â\x93the-data-vaultÂ\x94', '1', '0', 'markgainor1', '9/26/2016 3:14']




Now that we've removed the headers from hn, we're ready to filter our data. Since we're only concerned with post titles beginning with **Ask HN** or **Show HN**, we'll create new lists of lists containing just the data for those titles.

## Sorting the Data

To find the posts that begin with either Ask HN or Show HN (and case variations) , we'll use the string method startswith and check the data by printing five rows of each list.

In [ ]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1]
    title_lower = title.lower()
    
    if title_lower.startswith('ask hn'):
        ask_posts.append(row)
    
    elif title_lower.startswith('show hn'):
        show_posts.append(row)
        
    else:
        other_posts.append(row)
        
for row in ask_posts[:5]:
    print (row)
    print('\n')
print('\n')
for row in show_posts[:5]:
    print (row)
    print('\n')
print('\n')
for row in other_posts[:5]:
    print (row)
    print('\n')
print('Number of Ask HN posts:',len(ask_posts))
print('Number of Show HN posts:',len(show_posts))
print('Number of Other posts:',len(other_posts))

['12578908', 'Ask HN: What TLD do you use for local development?', '', '4', '7', 'Sevrene', '9/26/2016 2:53']


['12578522', 'Ask HN: How do you pass on your work when you die?', '', '6', '3', 'PascLeRasc', '9/26/2016 1:17']


['12577908', 'Ask HN: How a DNS problem can be limited to a geographic region?', '', '1', '0', 'kuon', '9/25/2016 22:57']


['12577870', 'Ask HN: Why join a fund when you can be an angel?', '', '1', '3', 'anthony_james', '9/25/2016 22:48']


['12577647', 'Ask HN: Someone uses stock trading as passive income?', '', '5', '2', '00taffe', '9/25/2016 21:50']




['12578335', 'Show HN: Finding puns computationally', 'http://puns.samueltaylor.org/', '2', '0', 'saamm', '9/26/2016 0:36']


['12578182', 'Show HN: A simple library for complicated animations', 'https://christinecha.github.io/choreographer-js/', '1', '0', 'christinecha', '9/26/2016 0:01']


['12578098', 'Show HN: WebGL visualization of DNA sequences', 'http://grondilu.github.io/dna.html', '1', '0', 'grondilu'

Next, let's determine if ask posts or show posts receive more comments on average.

In [ ]:
total_ask_comments = 0

for row in ask_posts:
    total_ask_comments +=  int(row[4])
    
avg_ask_comments = total_ask_comments / len (ask_posts)

total_show_comments = 0

for row in show_posts:
    total_show_comments +=  int(row[4])
    
avg_show_comments = total_show_comments / len (show_posts)

print('Average number of Ask HN comments:',avg_ask_comments)
print('Average number of Show HN comments:',avg_show_comments)

Average number of Ask HN comments: 10.393478498741656
Average number of Show HN comments: 4.886099625910612


## Reformatting and Analyzing the Data

It's clear that, on average, ask posts receive more comments than show posts. Since ask posts are more likely to receive comments, we'll focus our remaining analysis just on these posts.

Next, we'll determine if ask posts created at a certain time are more likely to attract comments. We'll use the following steps to perform this analysis:

1. Calculate the amount of ask posts created in each hour of the day, along with the number of comments received.
2. Calculate the average number of comments ask posts receive by hour created.

We'll use the datetime module to work with the data in the created_at column to calculate the amount of ask posts and comments by hour created.

In [ ]:
import datetime as dt

result_list = []

for row in ask_posts:
    result_list.append ( [ row[6] , int(row[4]) ] )
    
counts_by_hour = {}
comments_by_hour = {}

for row in result_list:
    time_created = dt.datetime.strptime(row[0],'%m/%d/%Y %H:%M')
    hour = time_created.hour
    
    if hour not in counts_by_hour:
        counts_by_hour[hour] = 1
        comments_by_hour[hour] = row[1]
    else:
        counts_by_hour[hour] += 1
        comments_by_hour[hour] += row[1]

print('The number of ask posts created during each hour of the day:')
print(counts_by_hour)
print('\n')
print('The corresponding number of comments ask posts created at each hour received:')
print(comments_by_hour)

The number of ask posts created during each hour of the day:
{2: 269, 1: 282, 22: 383, 21: 518, 19: 552, 17: 587, 15: 646, 14: 513, 13: 444, 11: 312, 10: 282, 9: 222, 7: 226, 3: 271, 23: 343, 20: 510, 16: 579, 8: 257, 0: 301, 18: 614, 12: 342, 4: 243, 6: 234, 5: 209}


The corresponding number of comments ask posts created at each hour received:
{2: 2996, 1: 2089, 22: 3372, 21: 4500, 19: 3954, 17: 5547, 15: 18525, 14: 4972, 13: 7245, 11: 2797, 10: 3013, 9: 1477, 7: 1585, 3: 2154, 23: 2297, 20: 4462, 16: 4466, 8: 2362, 0: 2277, 18: 4877, 12: 4234, 4: 2360, 6: 1587, 5: 1838}


Next, we'll use these two dictionaries to calculate the average number of comments for posts created during each hour of the day by creating a list of lists containing the hours during which posts were created and the average number of comments those posts received.

In [ ]:
avg_by_hour = []

for hour in counts_by_hour:
    no_of_posts = counts_by_hour[hour]
    no_of_comments = comments_by_hour[hour]
    avg_no_comments = comments_by_hour[hour] / counts_by_hour[hour] 
    avg_by_hour.append([hour,avg_no_comments])
    
print(avg_by_hour)

[[2, 11.137546468401487], [1, 7.407801418439717], [22, 8.804177545691905], [21, 8.687258687258687], [19, 7.163043478260869], [17, 9.449744463373083], [15, 28.676470588235293], [14, 9.692007797270955], [13, 16.31756756756757], [11, 8.96474358974359], [10, 10.684397163120567], [9, 6.653153153153153], [7, 7.013274336283186], [3, 7.948339483394834], [23, 6.696793002915452], [20, 8.749019607843136], [16, 7.713298791018998], [8, 9.190661478599221], [0, 7.5647840531561465], [18, 7.94299674267101], [12, 12.380116959064328], [4, 9.7119341563786], [6, 6.782051282051282], [5, 8.794258373205741]]


Now let's sort the above data in descending order of average number of comments for easy analysis.

In [ ]:
swap_avg_by_hour = []

for value in avg_by_hour:
    swap_avg_by_hour.append([value[1],value[0]])
    
sorted_swap = sorted(swap_avg_by_hour,reverse = True)

print('Top 5 Hours for Ask Posts Comments')
print('\n')

for row in sorted_swap:
    template = '{time}:00: {num:.2f} average comments per post.'
    print(template.format(time = row[1] , num = row[0]))

Top 5 Hours for Ask Posts Comments


15:00: 28.68 average comments per post.
13:00: 16.32 average comments per post.
12:00: 12.38 average comments per post.
2:00: 11.14 average comments per post.
10:00: 10.68 average comments per post.
4:00: 9.71 average comments per post.
14:00: 9.69 average comments per post.
17:00: 9.45 average comments per post.
8:00: 9.19 average comments per post.
11:00: 8.96 average comments per post.
22:00: 8.80 average comments per post.
5:00: 8.79 average comments per post.
20:00: 8.75 average comments per post.
21:00: 8.69 average comments per post.
3:00: 7.95 average comments per post.
18:00: 7.94 average comments per post.
16:00: 7.71 average comments per post.
0:00: 7.56 average comments per post.
1:00: 7.41 average comments per post.
19:00: 7.16 average comments per post.
7:00: 7.01 average comments per post.
6:00: 6.78 average comments per post.
23:00: 6.70 average comments per post.
9:00: 6.65 average comments per post.


# Conclusion

- The Ask HN posts has Higher Average number of comments than the Show HN posts.
- 3 pm GMT has the highest average number of comments.